In [3]:
import numpy as np
from tensorflow import keras
from tools import signature as sig
from sklearn.preprocessing import StandardScaler

from ipywidgets import interact, fixed
import ipywidgets as widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from ipywidgets import Image, Layout
import PIL.Image
import io

In [4]:
# Téléchargement des données
(x_train_origin, y_train_origin), (x_test_origin, y_test_origin)= \
        keras.datasets.mnist.load_data(path='all_mnist.npz')

In [5]:
keep = [1, 3 , 7]
keep.sort()
n = 100

keep.sort()
(x, y), (x_test, y_test) = sig.pretraitement(x_train_origin, y_train_origin, 
                                                 x_test_origin, y_test_origin, 
                                                 keep=keep, n=n, verbose=False)

In [6]:
model_name="./models/signature_all_100"
model = sig.trained_model(x, y, hidden_layers=[32, 16],
                          model_name=model_name, verbose=True,load=False)

predictions = sig.get_preditions(model, x, keep, verbose=False)

labels = [keep[np.argmax(l)] for l in y]

hidden_layers = model.get_hidden_layers_outputs(x)

for i in range(len(hidden_layers)):
    hidden_layers[i] = StandardScaler().fit_transform(hidden_layers[i])

best_k = sig.get_best_k(hidden_layers, kmax = len(keep),verbose=False)
clusters = sig.clustering(hidden_layers, best_k)



- input_shape:  (784,)
- n_neurons:  [32, 16, 3]
Epoch 1/15
25/25 - 1s - loss: 0.2104 - accuracy: 0.4933
Epoch 2/15
25/25 - 0s - loss: 0.1838 - accuracy: 0.7433
Epoch 3/15
25/25 - 0s - loss: 0.1607 - accuracy: 0.8433
Epoch 4/15
25/25 - 0s - loss: 0.1397 - accuracy: 0.9033
Epoch 5/15
25/25 - 0s - loss: 0.1211 - accuracy: 0.9500
Epoch 6/15
25/25 - 0s - loss: 0.1050 - accuracy: 0.9567
Epoch 7/15
25/25 - 0s - loss: 0.0910 - accuracy: 0.9667
Epoch 8/15
25/25 - 0s - loss: 0.0789 - accuracy: 0.9733
Epoch 9/15
25/25 - 0s - loss: 0.0687 - accuracy: 0.9733
Epoch 10/15
25/25 - 0s - loss: 0.0598 - accuracy: 0.9700
Epoch 11/15
25/25 - 0s - loss: 0.0521 - accuracy: 0.9700
Epoch 12/15
25/25 - 0s - loss: 0.0455 - accuracy: 0.9700
Epoch 13/15
25/25 - 0s - loss: 0.0401 - accuracy: 0.9733
Epoch 14/15
25/25 - 0s - loss: 0.0357 - accuracy: 0.9733
Epoch 15/15
25/25 - 0s - loss: 0.0320 - accuracy: 0.9733
INFO:tensorflow:Assets written to: ./models/signature_all_100/assets


In [7]:
dictionary_data = dict()

dictionary_data['x'] = x.tolist()
dictionary_data['predictions'] = predictions.tolist()
dictionary_data['hidden_layers'] = [e.tolist() for e in hidden_layers]
dictionary_data['clusters'] = [e.tolist() for e in clusters]
dictionary_data['labels'] = [int(e) for e in labels]
dictionary_data['keep'] = keep
dictionary_data['best_k'] = [int(e) for e in best_k]

sig.saveJson("./json/signature_137_100.json", dictionary_data)

In [8]:
hidden_layers[0][0]

array([-0.59129445, -0.12178362,  0.76050481, -0.34433027, -0.76420291,
       -1.23300715, -0.91393141, -0.71392852, -0.59035562, -0.87485081,
       -0.9657925 , -0.11164789, -1.26841968,  1.06966108, -0.08192214,
       -0.65367324,  1.30537999,  0.01059932, -0.45116781, -0.23895567,
       -0.86314508, -0.12159545,  0.2736382 , -0.09850347, -0.71676883,
        1.34588877, -0.26856725, -0.11735671,  1.7637605 ,  0.01682869,
        0.30756139, -1.46825185])

In [9]:
# Build colorscale
colorscale = {
    'color': np.zeros(len(x), dtype='uint8'),
    'colorscale': ['grey', '#ee1717', '#7201a8', '#17a60d', '#291ae0', '#f418ff', '#e3ae04', 
                    '#fb5e09', 'black', '#0ed6e1', '#d35400'],
    'cmin': 0,
    'cmax': 10
}

dict_data = sig.loadJson("./json/signature_137_100")
dict_data['hidden_layers'] = [np.array(e) for e in dict_data['hidden_layers']]

traces = sig.umap_plot(x, labels, predictions, hidden_layers, clusters, colorscale)

parcats = sig.parcats(labels, keep, best_k, clusters, predictions, colorscale)

traces.append(parcats)

In [10]:
dict_data['hidden_layers'][0][0]

array([-0.22757819, -0.79512683,  0.16782288,  0.61645832, -0.39777305,
       -0.73877508,  1.87092877, -0.1690636 , -0.75209623,  0.2067113 ,
        1.80121284, -0.39938427, -1.52332161, -0.84367227, -1.11410413,
       -0.72569969,  0.843273  , -1.14408849, -0.66837436,  0.        ,
       -0.93577334, -0.78200563,  1.5594385 , -0.29373152,  2.20142065,
        1.11667012, -0.5616308 , -0.00880984, -0.18264359, -2.06096712,
        0.9336958 , -0.2066273 ])

In [11]:
def image_to_byte_array(image):
    imgByteArr = io.BytesIO()
    image.save(imgByteArr, format='png')
    imgByteArr = imgByteArr.getvalue()
    return imgByteArr

def to_image(x):
    return  image_to_byte_array(
                PIL.Image.fromarray(
                    (x.reshape((len(x_train_origin[0]) , len(x_train_origin[0][0]))) * 255).astype(np.uint8)
                ))

# Update color callback
def update_color(trace, points, state):
    # Compute new color array
    new_color = np.array(trace.marker.color)
    new_color[points.point_inds] = color_toggle.index
    with fig.batch_update():
        # Update scatter color
        for i in range(len(fig.data)-1):
            fig.data[i].marker.color = new_color

        # Update parcats colors
        fig.data[len(fig.data)-1].line.color = new_color
        
# Update color callback
def update_parcats_color(trace, points, state):
    # Compute new color array
    new_color = np.array(trace.line.color)
    new_color[points.point_inds] = color_toggle.index
    with fig.batch_update():
        # Update scatter color
        for i in range(len(fig.data)-1):
            fig.data[i].marker.color = new_color

        # Update parcats colors
        fig.data[len(fig.data)-1].line.color = new_color
        
def print_image(trace, points, state):     
    # Image update
    if (points.point_inds != []):
        image_widget.value = to_image(x[points.point_inds[0]])

In [12]:
fig = make_subplots(rows=3, figure=go.FigureWidget(), specs=[[{'type': 'xy'}],
                                                             [{'type': 'xy'}],
                                                            [{'type': 'parcats'}]])

fig.add_traces(traces, rows=[1, 2, 3], cols=[1, 1, 1])

fig.update_layout(height=800,
                    dragmode='lasso', hovermode='closest')

# Build color selection widget
color_toggle = widgets.ToggleButtons(
    options=['None', 'Red', 'Purple', 'Green', 'Blue', 'Pink', 
                 'Yellow', 'Orange','Black' ,'Aqua', 'Brown'],
    index=1, description='Color:', disabled=False)
    
image_widget = Image(
    value=to_image(x[0]),
    layout=Layout(height='100px', width='100px')
)

# Register callback on scatter selection...
fig.data[0].on_selection(update_color)
fig.data[1].on_selection(update_color)

fig.data[0].on_hover(print_image)
fig.data[1].on_hover(print_image)

# and parcats click
fig.data[2].on_click(update_parcats_color)

# Display figure
widgets.VBox([color_toggle, image_widget, fig])

In [13]:
display(fig)

FigureWidget({
    'data': [{'customdata': array([[1, 1, 2],
                                   [1, 1, 2],
   …